In this notebook, the goal is two folds:
1. Model the different covariates from the text only
2. Then model the outcome given the predicted covariates & compare this with model build on the true covariates

An important consideration is that we want the split to be the same across all notebooks, we save this information to be sure to be consistent across all experiments.

In [ ]:
import os
import numpy as np
import pandas as pd 

In [ ]:
outcomes  = pd.read_csv('data/TGCA_Merged.csv', index_col = 0)

### Extract target labels

In [ ]:
# TODO: Adapt this with the structured columns you would like to predict from the unstructured data
outcomes_to_predict = outcomes[['type', 'gender', 'race', 'ajcc_pathologic_tumor_stage']]
outcomes_to_predict['ajcc_pathologic_tumor_stage'] = outcomes_to_predict.ajcc_pathologic_tumor_stage.astype('category')
outcomes_to_predict_dummy = pd.get_dummies(outcomes_to_predict, dummy_na = True).astype(int) # Binarisation

In [ ]:
outcomes_to_predict_dummy.to_csv('data/binary_embedding.csv')

### Split the data 

We propose two different 3 fold cross validation evaluation based on:
1. Hospital out evaluation: to ensure that one hospital is only one fold.
2. Cancer group: to evaluate how the performance on a suubset of cancer types generalise.

The following save how the data should be evaluated.

In [ ]:
# TODO: Adapt this split given the columns of your dataset.
split = pd.DataFrame({
        'Hospital': np.nan,
        'Grouping' : np.nan,
    }, index = outcomes.index)

In [ ]:
from sklearn.model_selection import GroupKFold
for split_column in split:
    kf = GroupKFold(n_splits = 3)
    for i, (_, test_index) in enumerate(kf.split(outcomes, groups = outcomes[split_column])):
        split.iloc[test_index] = i
split.to_csv('results/split.csv')

Run one of the section Embedding, Fine-Tuning or Promptin

-----------

# Embedding

From extracted embedding from LLM try to predict the different covariates of interest.

### Open embedding

In [ ]:
# Embedding to use for the notebook
embedding_type = 'BERT' # BERT, clinicalBERT

In [ ]:
embedding = pd.read_csv('data/{}_embedding.csv'.format(embedding_type), index_col = 0)

In [ ]:
assert (outcomes.index == embedding.index).all(), 'Misaligned index may create an issue - How is the embedding obtained?'

In [ ]:
outcomes.head()

In [ ]:
embedding.head()

### Model the different covariates

We aim to predict from the text each manually extracted covariates. 

Then we save these covariates for future predictions.

In [ ]:
# For simplicity, we rely on a NN from sklearn for this task
from sklearn.neural_network import MLPClassifier

In [ ]:
predictions = {}
for split_type in split.columns:
    predictions[split_type] = pd.DataFrame().reindex_like(outcomes_to_predict_dummy)
    for fold in split[split_type].dropna().unique():
        train = split[split_type].values != fold
        test = split[split_type].values == fold

        model = MLPClassifier(hidden_layer_sizes = [], random_state = 42, 
                              learning_rate_init = 0.01, max_iter = 100, 
                              early_stopping = True).fit(embedding[train].values, outcomes_to_predict_dummy[train].values)
        predictions[split_type][test] = model.predict_proba(embedding[test].values)

In [ ]:
predictions = pd.concat(predictions)

In [ ]:
assert False, "Go to Section 'Binarise and Save' to save this extraction."

--------

# Fine - Tuning

Be careful this will overwrite the previous Section (jump to last to save and evaluate).
In this Section, we aim to fine-tune a neural network to extract the concept of interest.

In [ ]:
embedding_type = 'clinicalBERT' # clinicalBERT, BERT
embedding_type += '_finetune' # For naming distinction

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

In [ ]:
assert torch.cuda.is_available(), 'Machine or configuration not using GPU'

In [ ]:
def get_model(embedding):
    if embedding == 'BERT_finetune':
        from transformers import BertTokenizer, BertForSequenceClassification
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = outcomes_to_predict_dummy.shape[1],
            output_attentions = False, output_hidden_states = False, problem_type="multi_label_classification")
    elif embedding == 'clinicalBERT_finetune':
        from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
        tokenizer = DistilBertTokenizer.from_pretrained('medicalai/ClinicalBERT')
        model = DistilBertForSequenceClassification.from_pretrained("medicalai/ClinicalBERT", num_labels = outcomes_to_predict_dummy.shape[1],
            output_attentions = False, output_hidden_states = False, problem_type="multi_label_classification")

    return tokenizer, model

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
training_args = TrainingArguments(output_dir = 'results/', num_train_epochs = 10)

In [ ]:
predictions = {}
for split_type in split.columns:
    predictions[split_type] = pd.DataFrame().reindex_like(outcomes_to_predict_dummy)
    for fold in split[split_type].dropna().unique():
        train = split[split_type].values != fold
        test = split[split_type].values == fold

        # Load model and encode data
        tokenizer, model = get_model(embedding_type)
        train_endcoded = Dataset(tokenizer(outcomes[train].text.tolist(), truncation = True, padding = True), outcomes_to_predict_dummy[train])
        test_endcoded = Dataset(tokenizer(outcomes[test].text.tolist(), truncation = True, padding = True), outcomes_to_predict_dummy[test])

        # Train model
        trainer = Trainer(model = model, args = training_args, 
                          train_dataset = train_endcoded)
        trainer.train()

        # Predict
        predictions[split_type][test] = trainer.predict(test_endcoded).predictions

In [ ]:
predictions = pd.concat(predictions)

In [ ]:
assert False, "Go to Section 'Binarise and Save' to save this extraction."

-----------

# Prompting

In [ ]:
from transformers import pipeline

In [ ]:
model = pipeline("text-generation", model="medalpaca/medalpaca-7b", tokenizer="medalpaca/medalpaca-7b") # Should be downloaded in my folder (10 Gb)

In [ ]:
outcomes_match = {
    "type": ("cancer_type", outcomes_to_predict["type"].dropna().unique()),
    "gender": ('sex', ['male', 'female']),
    "race": ('ethnicity', ['white', 'non-white']),
    "ajcc_pathologic_tumor_stage": ('Cancer stage', ['I', 'II', 'III'])
}

In [ ]:
binarised_predictions = pd.DataFrame().reindex_like(outcomes_to_predict)
for outcome in outcomes_to_predict:
    name, values = outcomes_match[outcome]
    for patient in binarised_predictions.index:
        patient_text = outcomes.text[patient]
        prompt = "Context: Pathology report {}\n\n Question: Based on the provided pathology report, what is the {} (possible values: {})? Please provide your answer as one of these values, without any additional text or explanations.\n\nAnswer:".format(patient_text, name, values)
        binarised_predictions.loc[patient, outcome] = model(prompt)

In [ ]:
binarised_predictions.to_csv('data/alpaca_predicted_binary.csv')

In [ ]:
assert False, "You are done ! This method does not require binarisations."

-------

# Discretise and save

As the previous sections predict probabilities, we then discretise the outputs to match the concept of interest.

In [ ]:
# Binarisation by a softmax
binarised_predictions = []
for column in outcomes_to_predict.columns:
    if column == 'ajcc_pathologic_tumor_stage': # Special float case
        pred_col = predictions.loc[:, predictions.columns.str.contains(column)].idxmax(axis = 1).str.replace(column + '_', '').astype(float)
    elif column == 'type': # Special categorial case (change for other dataset)
        pred_col = predictions.loc[:, predictions.columns.str.contains(column)].idxmax(axis = 1).str.replace(column + '_', '')
    else: # All other binary variable
        pred_col = predictions.loc[:, column] > 0.5

    binarised_predictions.append(pred_col.rename(column))
binarised_predictions = pd.concat(binarised_predictions, axis = 1)
binarised_predictions.head()

In [ ]:
binarised_predictions.to_csv('data/{}_predicted_binary.csv'.format(embedding_type))

You have now saved the predicted binary outcome.

----------

### Measure performance of the extraction

This section allows you to check the performance for extracting the concetps of interest. Maybe the text does not contain the information or any indicator of the conctept you aim to extract.

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
performance = {}
for split_type in split.columns:
    columns = split[split_type].dropna().unique()
    performance[split_type] = pd.DataFrame(index = columns, columns = predictions.columns)
    for fold in columns:
        for dimension in predictions.columns:
            test = split[split_type] == fold
            mean = outcomes_to_predict_dummy.loc[test, dimension].mean()
            if mean != 1 and mean != 0:
                # The class contains some positive
                performance[split_type].loc[fold, dimension] = roc_auc_score(outcomes_to_predict_dummy.loc[test, dimension], predictions.loc[(split_type, test[test].index), dimension])
performance = pd.concat(performance)

In [ ]:
performance.loc['Hospital'].astype(float).describe()

In [ ]:
performance.loc['Grouping'].astype(float).describe()